## 데이터 불러오기

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline


In [9]:
from tsai.all import *
X, y, splits = get_UCR_data('HandMovementDirection', split_data=False)
print(X.shape)
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_sample=True)]
mv_clf = TSClassifier(X, y, splits=splits, path='models', arch=InceptionTimePlus, batch_tfms=batch_tfms, metrics=accuracy, cbs=ShowGraph())
dls = get_ts_dls(X, y, splits=splits, tfms=tfms, drop_last=False, shuffle_train=False, batch_tfms=batch_tfms, bs=10_000)

(234, 10, 400)


(234, 10, 400)

# 모델 생성

### nrmse score metric

In [13]:
from sklearn.metrics import mean_squared_error
import tensorflow as tf
tf.random.set_seed(42)

def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(14): # ignore 'ID'
        rmse = mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:7]) + 1.0 * np.sum(all_nrmse[7:14])
    return score

## Custom multiouputRegressor
for use early stopping & model save, load

In [17]:
from sklearn.utils.validation import _check_fit_params
from sklearn.base import is_classifier
from sklearn.utils.fixes import delayed
from joblib import Parallel
from sklearn.multioutput import _fit_estimator
from sklearn.multioutput import MultiOutputRegressor
import lightgbm

class MyMultiOutputRegressor_LGBM(MultiOutputRegressor):
    def fit(self, X, y, sample_weight=None, **fit_params):
        """ Fit the model to data.
        Fit a separate model for each output variable.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data.
        y : {array-like, sparse matrix} of shape (n_samples, n_outputs)
            Multi-output targets. An indicator matrix turns on multilabel
            estimation.
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.
            Only supported if the underlying regressor supports sample
            weights.
        **fit_params : dict of string -> object
            Parameters passed to the ``estimator.fit`` method of each step.
            .. versionadded:: 0.23
        Returns
        -------
        self : object
        """

        if not hasattr(self.estimator, "fit"):
            raise ValueError("The base estimator should implement"
                             " a fit method")

        X, y = self._validate_data(X, y,
                                   force_all_finite=False,
                                   multi_output=True, accept_sparse=True)

        if is_classifier(self):
            check_classification_targets(y)

        if y.ndim == 1:
            raise ValueError("y must have at least two dimensions for "
                             "multi-output regression but has only one.")

        if (sample_weight is not None and
                not has_fit_parameter(self.estimator, 'sample_weight')):
            raise ValueError("Underlying estimator does not support"
                             " sample weights.")
            
        lambda_y = lambda i: '0'+str(i+1) if i<9 else str(i+1)

        fit_params_validated = _check_fit_params(X, fit_params)
        [(X_test, Y_test)] = fit_params_validated.pop('eval_set')
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_estimator)(
                self.estimator, X, y[:, i], sample_weight,
                **fit_params_validated, 
                eval_set=[(X_test, Y_test[:, i])], 
                eval_names="Y_" + lambda_y(i),
                verbose=-1, 
                callbacks=[lightgbm.early_stopping(200)])
            for i in range(y.shape[1]))
        return self
    
    # model save
    def save(self, path):
        for chain_idx, estimator in enumerate(self.estimators_):
            save_path = '{}_{}.txt'.format(path, chain_idx)
            estimator.booster_.save_model(save_path)
    
    # model load
    def load(self, path):
        for chain_idx, estimator in enumerate(self.estimators_):
            save_path = '{}_{}.txt'.format(path, chain_idx)
            self.estimators_[chain_idx] = lightgbm.Booster(save_path)

In [18]:
from sklearn.utils.validation import _check_fit_params
from sklearn.base import is_classifier
from sklearn.utils.fixes import delayed
from joblib import Parallel
from sklearn.multioutput import _fit_estimator
from sklearn.multioutput import MultiOutputRegressor
import xgboost
import pickle

class MyMultiOutputRegressor_XGB(MultiOutputRegressor):
    
    def fit(self, X, y, sample_weight=None, **fit_params):
        """ Fit the model to data.
        Fit a separate model for each output variable.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Data.
        y : {array-like, sparse matrix} of shape (n_samples, n_outputs)
            Multi-output targets. An indicator matrix turns on multilabel
            estimation.
        sample_weight : array-like of shape (n_samples,), default=None
            Sample weights. If None, then samples are equally weighted.
            Only supported if the underlying regressor supports sample
            weights.
        **fit_params : dict of string -> object
            Parameters passed to the ``estimator.fit`` method of each step.
            .. versionadded:: 0.23
        Returns
        -------
        self : object
        """

        if not hasattr(self.estimator, "fit"):
            raise ValueError("The base estimator should implement"
                             " a fit method")

        X, y = self._validate_data(X, y,
                                   force_all_finite=False,
                                   multi_output=True, accept_sparse=True)

        if is_classifier(self):
            check_classification_targets(y)

        if y.ndim == 1:
            raise ValueError("y must have at least two dimensions for "
                             "multi-output regression but has only one.")

        if (sample_weight is not None and
                not has_fit_parameter(self.estimator, 'sample_weight')):
            raise ValueError("Underlying estimator does not support"
                             " sample weights.")

        fit_params_validated = _check_fit_params(X, fit_params)
        [(X_test, Y_test)] = fit_params_validated.pop('eval_set')
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_estimator)(
                self.estimator, X, y[:, i],
                eval_set=[(X_test, Y_test[:, i])],
                early_stopping_rounds=100, 
                #eval_metric = ["rmse"], 
                verbose=3000)
            
            for i in range(y.shape[1]))
        return self
    
    # model save
    def save(self, path):
        for chain_idx, estimator in enumerate(self.estimators_):
            save_path = '{}_{}.dat'.format(path, chain_idx)
            pickle.dump(estimator, open(save_path, "wb"))
    
    # model load
    def load(self, path):
        for chain_idx, estimator in enumerate(self.estimators_):
            save_path = '{}_{}.dat'.format(path, chain_idx)
            model = pickle.load(open(save_path, "rb"))


# 모델 학습
(CAT + XGB + LGB) with 6 CV

In [19]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
import lightgbm
import time
import warnings
import os
warnings.filterwarnings(action='ignore', category=UserWarning)

skf = StratifiedKFold(n_splits = 6, random_state = 42, shuffle = True) #총 6번의 fold 진행
n = 0 #x번째 fold인지 기록

fold_target_pred = []
fold_score = []

drop_x = ["X_02", "X_10","X_11", "X_34", "X_35", "X_36", "X_37","X_45"]

#파일 디렉토리 생성
model_dir = f'./model'
os.makedirs(model_dir, exist_ok=True)

for train_index, valid_index in skf.split(data_train_X, data_train_y['label']): #label 기준으로 stratified k fold 진행
    n += 1
    
    val_pred_name = [] #validation pred model 이름 저장
    val_pred = []      #validation set pred 결과 저장
    target_pred = []   #test set pred 결과 저장
    
    train_X = np.array(data_train_X.drop(drop_x, axis = 1))
    train_Y = np.array(data_train_y.drop(['label'], axis = 1)) # 분배된 학습을 위해 만들어뒀던 label feature 제거
    label_Y = np.array(data_train_y['label']) # label값만 가지는 데이터
    
    X_train, X_valid = train_X[train_index], train_X[valid_index]
    y_train, y_valid = train_Y[train_index], train_Y[valid_index]
    
    y_label = label_Y[train_index]

    X_test = np.array(data_test.drop(drop_x, axis = 1))
    
    ### Create Model ###
    #CAT model
    start_time_cat = time.time()
    model_cat = CatBoostRegressor(verbose = 200,
                            learning_rate = 0.02,
                            random_seed = 42,
                            iterations = 5000,
                            loss_function='MultiRMSE',
                            #ignored_features = [8, 9, 31, 32, 33, 34, 45, 50, 51, 53, 54, 55],
                            od_wait = 200,
                            depth = 9)
    
    model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    end_time_cat = time.time()
    
    
    #model cat 저장
    cat_path = './model/cat_{}'.format(n)
    model_cat.save_model(cat_path)
    
    #model cat 불러오기
    #model_cat.load_model(cat_path)
    
    val_pred_name.append("CatBoostRegressor")  # 모델 이름 저장
    val_pred.append(model_cat.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model_cat.predict(X_test)) # test set pred 결과 저장
    
    ### LGBM model
    start_time_lgb = time.time()
    model_lgbm = MyMultiOutputRegressor_LGBM(LGBMRegressor(n_estimators = 2000, 
                                               learning_rate = 0.01,
                                               max_depth = 16,
                                               min_child_samples = 56,
                                               subsample = 0.4,
                                               num_leaves = 160,
                                               random_state = 42,
                                               n_jobs = 8))

    fit_params = dict(
        eval_set=[(X_valid, y_valid)],
        eval_metric = "rmse",
        )

    model_lgbm.fit(X_train, y_train, **fit_params)
    end_time_lgb = time.time()
    val_pred_name.append("LGBMRegressor")  # 모델 이름 저장
    val_pred.append(model_lgbm.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model_lgbm.predict(X_test)) # test set pred 결과 저장
    
    #model lgbm 저장
    lgbm_path = './model/lgbm_{}'.format(n)
    model_lgbm.save(lgbm_path)
    
    #model lgbm 불러오기
    #model_lgbm.load(lgbm_path)

    ### XGB model
    start_time_xgb = time.time()
    model_xgb = MyMultiOutputRegressor_XGB(XGBRegressor(objective = "reg:squarederror",
                                                  n_estimators = 3000,
                                                  random_state = 42,
                                                  eval_metric = "rmse", 
                                                  learning_rate=0.006,
                                                  subsample=0.75, 
                                                  colsample_bytree = 0.86,
                                                  max_depth=9,
                                                  tree_method='gpu_hist', 
                                                  gpu_id = 0))
    
    fit_params = dict(
        eval_set=[(X_valid, y_valid)],
        )
    
    model_xgb.fit(X_train, y_train, **fit_params)
    end_time_xgb = time.time()
    val_pred_name.append("XGBRegressor")  # 모델 이름 저장
    val_pred.append(model_xgb.predict(X_valid))   # validation set pred 결과 저장
    target_pred.append(model_xgb.predict(X_test)) # test set pred 결과 저장
    
    #model xgb 저장
    xgb_path = './model/xgb_{}'.format(n)
    model_xgb.save(xgb_path)
    
    #model xgb 불러오기
    #model_xgb.load(xgb_path)
  
    ### average validation pred ###
    preds = np.array(val_pred[0])
    for i in range(1, len(val_pred)):
        preds += val_pred[i]
    preds = preds/len(val_pred)

    ### average target pred ###
    target_preds = target_pred[0]
    for i in range(1, len(target_pred)):
        target_preds += target_pred[i]
    target_preds = target_preds/len(target_pred)
    fold_target_pred.append(target_preds) # append final target pred
    
    print("========== fold %d ==========" %(n))
    for i in range(len(val_pred)):
        print("%s model nrmse : %0.4f" %(val_pred_name[i], lg_nrmse(y_valid, val_pred[i])))
        
    print('CAT 코드 실행 시간: %10ds' % (end_time_cat - start_time_cat))
    print('LGB 코드 실행 시간: %10ds' % (end_time_lgb - start_time_lgb))
    print('XGB 코드 실행 시간: %10ds' % (end_time_xgb - start_time_xgb))
    print("average model nrmse : %0.4f" %(lg_nrmse(y_valid, preds)))
    fold_score.append(lg_nrmse(y_valid, preds))

total_score = fold_score[0]
for i in range(1, len(fold_score)):
    total_score += fold_score[i]
    
total_score = total_score/len(fold_score)

print("==============================")
print("Model Sum Average nrmse %0.4f" %(total_score))

0:	learn: 4.6352787	test: 4.6553672	best: 4.6553672 (0)	total: 227ms	remaining: 18m 54s
200:	learn: 4.3642074	test: 4.5361587	best: 4.5361587 (200)	total: 35.7s	remaining: 14m 12s
400:	learn: 4.2296767	test: 4.5122390	best: 4.5122182 (399)	total: 1m 10s	remaining: 13m 25s
600:	learn: 4.1193230	test: 4.4969338	best: 4.4969338 (600)	total: 1m 44s	remaining: 12m 47s
800:	learn: 4.0108089	test: 4.4846937	best: 4.4846622 (798)	total: 2m 18s	remaining: 12m 7s
1000:	learn: 3.9048064	test: 4.4747544	best: 4.4747025 (988)	total: 2m 53s	remaining: 11m 32s
1200:	learn: 3.8093561	test: 4.4687731	best: 4.4687731 (1200)	total: 3m 27s	remaining: 10m 57s
1400:	learn: 3.7193387	test: 4.4639210	best: 4.4637640 (1397)	total: 4m 2s	remaining: 10m 21s
1600:	learn: 3.6299716	test: 4.4606051	best: 4.4606001 (1593)	total: 4m 36s	remaining: 9m 47s
1800:	learn: 3.5433574	test: 4.4583479	best: 4.4582837 (1791)	total: 5m 11s	remaining: 9m 13s
2000:	learn: 3.4615257	test: 4.4561844	best: 4.4559083 (1985)	total: 5m

# 결과 제출
6 cv soft voting

In [20]:
final_pred = np.array(fold_target_pred[0])

for i in range(1, 6):
    final_pred += fold_target_pred[i]

final_pred = final_pred/6

In [21]:
final_pred

array([[  1.43650797,   1.24628275,   1.12990453, ..., -25.94147077,
        -25.93628909, -25.98553301],
       [  1.50875523,   1.23834255,   1.15887145, ..., -26.11785189,
        -26.14873015, -26.13291365],
       [  1.49355102,   1.18278801,   1.12252219, ..., -25.81403033,
        -25.7876382 , -25.7702005 ],
       ...,
       [  1.23594559,   0.91828441,   0.97499806, ..., -26.4815955 ,
        -26.4747848 , -26.50836623],
       [  1.20143511,   0.84251556,   0.92003249, ..., -26.41303087,
        -26.42648215, -26.45366266],
       [  1.29571369,   0.97338884,   1.0084016 , ..., -26.57056884,
        -26.55470405, -26.57788576]])

### Submission

In [22]:
filename = './sample_submission.csv'
sample_submission = pd.read_csv(filename)
sample_submission.head(5)

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TEST_00002,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,TEST_00003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TEST_00004,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,TEST_00005,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
for idx, col in enumerate(sample_submission.columns):
    if col=='ID':
        continue
    sample_submission[col] = final_pred[:,idx-1]
sample_submission.head()

,ID,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,TEST_00001,1.436508,1.246283,1.129905,14.188473,31.280681,16.680478,3.115658,-26.008348,-26.026714,-22.132417,24.607211,-25.941471,-25.936289,-25.985533
1,TEST_00002,1.508755,1.238343,1.158871,13.063973,30.682600,16.515653,3.213386,-26.186201,-26.179164,-22.438306,24.230160,-26.117852,-26.148730,-26.132914
2,TEST_00003,1.493551,1.182788,1.122522,14.668821,32.125480,16.925155,3.028501,-25.838927,-25.844483,-21.944534,24.660105,-25.814030,-25.787638,-25.770201
3,TEST_00004,1.446701,1.189539,1.057934,15.040722,32.612210,17.226454,3.075741,-25.633640,-25.650345,-21.715864,25.009979,-25.607663,-25.609216,-25.620737
4,TEST_00005,1.360789,1.007224,0.961567,14.810293,31.456541,16.940240,3.164222,-25.658716,-25.691876,-22.107019,24.702687,-25.598848,-25.599207,-25.599415


In [24]:
sample_submission.to_csv('./final_pred.csv', index=False)

In [26]:
import glob
model_paths = sorted(glob.glob('./model/*'))

import zipfile
submission = zipfile.ZipFile("./model.zip", 'w')
for path in model_paths:
    submission.write(path)
submission.close()